In [0]:
#define widgets -- NEEDS TO DEFINE IT ONCE
#dbutils.widgets.text("environment", "","")

#dynamic variables (pass it from ADF)
environment = dbutils.widgets.get("environment")

In [0]:
%run ../bi_config/lib_helper

In [0]:
%run ../bi_config/lib_sentry

In [0]:
#============================= Global Variable Declaration ===================================#
from datetime import datetime
import json

bronze_folder_path = "/mnt/"+ environment + "/bronze"
silver_folder_path = "/mnt/"+ environment + "/silver"
gold_folder_path = "/mnt/"+ environment + "/gold"

b_jobruninfo_path = gold_folder_path + "/jobruninfo"


In [0]:
#============================= Update Job Run Info ===================================#
from pyspark.sql.functions import col,concat,lit,current_date
from delta.tables import *
from pyspark.sql.types import StringType,TimestampType, LongType 
import hashlib
from datetime import date, timedelta, datetime
import os

notebook = os.path.basename(getNotebookPath())

try: 
  #read bronze table for further processing 
  deltaJobRunInfo = DeltaTable.forPath(spark, b_jobruninfo_path) 
  end_datetime = datetime.strftime(datetime.utcnow(),'%Y-%m-%d %H:%M:%S')
 
  df_job_info = spark.read.format("delta").load(b_jobruninfo_path)  
  df_batch_id = df_job_info.agg({"batch_id" : "max"}).collect()[0]
  df_batch_start_datetime = df_job_info.agg({"batch_start_datetime" : "max"}).collect()[0]
 
  deltaJobRunInfo.update(col("status")== "running", 
  {"status" : lit("success"),"end_datetime" : lit(end_datetime),"last_updated_datetime":lit(df_batch_start_datetime["max(batch_start_datetime)"])})
  
except Exception as error:
  log_error("{} {}".format(notebook, error)) #log error in sentry
  raise dbutils.notebook.exit(error) #raise the exception